# 🚀 Transformer Training & Fine-Tuning Notebook

**Professional ML Training Environment** for transformer models exported from [Transformer Builder](https://transformer-builder.com).

## Quick Start Modes

| Mode | Epochs | Time | Use Case |
|------|--------|------|----------|
| **⚡ Fast** | 3 | ~5 min | Quick validation |
| **⚖️ Balanced** | 10 | ~15 min | Development |
| **💎 Quality** | 20 | ~45 min | Production |

## Features
- ✅ 5 Data Sources (HuggingFace, Drive, Upload, Local, Synthetic)
- ✅ Live Training Visualization
- ✅ Google Drive Checkpoints
- ✅ W&B + Local SQLite Tracking
- ✅ Hyperparameter Search
- ✅ Export & Comparison Tools

**📌 Tip**: Run all cells in order for best results. Adjust hyperparameters in Section 3.

## 📋 Table of Contents

1. [Section 0: Quick Start](#section-0) ← You are here
2. [Section 1: Setup & Drive Workspace](#section-1) (2 min)
3. [Section 2: Model Loading](#section-2) (Load custom or example model)
4. [Section 3: Data Loading](#section-3) (5 sources)
5. [Section 4: Training Configuration](#section-4) (Hyperparameters)
6. [Section 5: W&B Tracking Setup](#section-5) (Optional)
7. [Section 6: Training Loop](#section-6) (Main training)
8. [Section 7: Analysis & Visualization](#section-7) (Dashboards)
9. [Section 8: Export & Results](#section-8) (Download checkpoints)
10. [Section 9: Advanced Features](#section-9) (Hyperparameter search)

⏱️ **Total Time**: ~20-60 minutes depending on mode


## 📦 Requirements

This notebook requires:
- Python >= 3.10
- PyTorch (pre-installed in Colab)
- Transformer Builder utilities (auto-downloaded)

**GPU Recommended** but not required. Training will auto-detect and use GPU if available.

---
<a id="section-1"></a>

In [ ]:
# Install training dependencies
!pip install -q -r https://raw.githubusercontent.com/matt-hans/transformer-builder-colab-templates/main/requirements-training.txt

# Check for Flash Attention support (PyTorch 2.0+)
import torch
print("✅ Dependencies installed")
print(f"   PyTorch version: {torch.__version__}")

if torch.__version__ >= "2.0.0":
    print("   ✅ PyTorch 2.0+ detected - Flash Attention support available")
    if torch.cuda.is_available():
        print("   ✅ CUDA available - Flash Attention will be enabled automatically")
    else:
        print("   ⚠️  CPU only - Flash Attention requires CUDA (GPU)")
else:
    print("   ⚠️  PyTorch < 2.0 - Flash Attention will be disabled")

In [ ]:
import os

print("📥 Downloading training utilities...")

# Remove old utils directory if exists
!rm -rf utils/

# Download complete utils package from GitHub
!git clone --depth 1 --branch main https://github.com/matt-hans/transformer-builder-colab-templates.git temp_repo 2>/dev/null

# Copy utils directory
!cp -r temp_repo/utils ./

# Cleanup
!rm -rf temp_repo

# Verify package structure
utils_path = os.path.join(os.getcwd(), 'utils')
if os.path.exists(utils_path):
    print(f"✅ Utils package downloaded")
    
    # Verify training subdirectory
    training_path = os.path.join(utils_path, 'training')
    if os.path.exists(training_path):
        n_files = len([f for f in os.listdir(training_path) if f.endswith('.py')])
        print(f"✅ Training utilities: {n_files} modules found")
    
    # Verify tier3 utilities
    tier3_path = os.path.join(utils_path, 'tier3_training_utilities.py')
    if os.path.exists(tier3_path):
        print(f"✅ Tier 3 training utilities ready")
else:
    print("❌ Failed to download utils package")
    raise RuntimeError("Could not download training utilities")

print("\n🔍 Verifying download...")

# Check critical modules exist
required_modules = [
    'utils/__init__.py',
    'utils/training/task_spec.py',
    'utils/training/training_config.py',
    'utils/training/training_core.py',
    'utils/training/metrics_tracker.py',
    'utils/training/drift_metrics.py',
    'utils/training/dashboard.py',
]

missing = []
for module in required_modules:
    if not os.path.exists(module):
        missing.append(module)

if missing:
    print("❌ Download incomplete! Missing modules:")
    for m in missing:
        print(f"   - {m}")
    print("\n" + "="*70)
    print("FALLBACK INSTRUCTIONS")
    print("="*70)
    print("\nManual download steps:")
    print("1. Run in a new cell:")
    print("   !git clone https://github.com/matt-hans/transformer-builder-colab-templates.git")
    print("   !cp -r transformer-builder-colab-templates/utils ./")
    print("   !rm -rf transformer-builder-colab-templates")
    print("\n2. Re-run this cell to verify")
    raise RuntimeError("Utils package download failed")

print("✅ All required modules verified")

In [ ]:
# @title 💾 Storage Configuration { display-mode: "form" }

from google.colab import drive
import os

# Storage option (user can choose)
storage_type = "Google Drive"  #@param ["Google Drive", "Local (session only)"]

print("=" * 70)
print("STORAGE CONFIGURATION")
print("=" * 70)
print()

workspace_root = None

if storage_type == "Google Drive":
    print("📂 Attempting to mount Google Drive...")
    print()
    
    try:
        # Try to mount Google Drive
        drive.mount('/content/drive', force_remount=False)
        
        # Create workspace folders on Drive
        workspace_root = '/content/drive/MyDrive/TransformerTraining'
        os.makedirs(f'{workspace_root}/checkpoints', exist_ok=True)
        os.makedirs(f'{workspace_root}/configs', exist_ok=True)
        os.makedirs(f'{workspace_root}/results', exist_ok=True)
        os.makedirs(f'{workspace_root}/datasets', exist_ok=True)
        
        print("✅ Google Drive mounted successfully!")
        print(f"✅ Workspace created at: {workspace_root}")
        print()
        print("📁 Directory structure:")
        print(f"   📁 checkpoints/ - Saved model weights")
        print(f"   📁 configs/ - Training configurations")
        print(f"   📁 results/ - Metrics, plots, dashboards")
        print(f"   📁 datasets/ - Cached datasets")
        print()
        print("💡 Benefits:")
        print("   • Files persist across sessions")
        print("   • Access from any device")
        print("   • Automatic backup")
        
    except Exception as e:
        print("❌ Google Drive mount failed!")
        print()
        print(f"Error: {e}")
        print()
        print("=" * 70)
        print("TROUBLESHOOTING")
        print("=" * 70)
        print()
        print("Common solutions:")
        print("  1. Click the authentication link that appears above")
        print("  2. Sign in with your Google account")
        print("  3. Grant permissions to access Google Drive")
        print("  4. If in a corporate environment, check with IT")
        print()
        print("=" * 70)
        print("FALLBACK: Using local storage")
        print("=" * 70)
        print()
        
        # Fallback to local storage
        workspace_root = '/content/workspace'
        os.makedirs(f'{workspace_root}/checkpoints', exist_ok=True)
        os.makedirs(f'{workspace_root}/configs', exist_ok=True)
        os.makedirs(f'{workspace_root}/results', exist_ok=True)
        os.makedirs(f'{workspace_root}/datasets', exist_ok=True)
        
        print(f"✅ Local workspace created at: {workspace_root}")
        print()
        print("⚠️  IMPORTANT: Local storage limitations:")
        print("   • Files will be DELETED when runtime ends")
        print("   • Maximum 12-hour session lifetime")
        print("   • Use 'Download results' option before session ends")

else:
    # User explicitly chose local storage
    print("📂 Using local storage (session only)...")
    print()
    
    workspace_root = '/content/workspace'
    os.makedirs(f'{workspace_root}/checkpoints', exist_ok=True)
    os.makedirs(f'{workspace_root}/configs', exist_ok=True)
    os.makedirs(f'{workspace_root}/results', exist_ok=True)
    os.makedirs(f'{workspace_root}/datasets', exist_ok=True)
    
    print(f"✅ Local workspace created at: {workspace_root}")
    print()
    print("📁 Directory structure:")
    print(f"   📁 checkpoints/ - Saved model weights")
    print(f"   📁 configs/ - Training configurations")
    print(f"   📁 results/ - Metrics, plots, dashboards")
    print(f"   📁 datasets/ - Cached datasets")
    print()
    print("⚠️  IMPORTANT: Local storage limitations:")
    print("   • Files will be DELETED when runtime ends")
    print("   • Maximum 12-hour session lifetime")
    print("   • Use Section 8 'Download results' to save locally")
    print()
    print("💡 Tip: Switch to 'Google Drive' above for persistent storage")

print()
print("=" * 70)
print("✅ STORAGE READY")
print("=" * 70)
print()
print(f"Workspace: {workspace_root}")
print()

In [ ]:
from utils.training.experiment_db import ExperimentDB

# Initialize local SQLite tracking (backup to W&B)
db = ExperimentDB(f'{workspace_root}/experiments.db')

print("✅ Experiment database initialized")
print(f"   Database: {workspace_root}/experiments.db")
print(f"   Recent runs:")
recent_runs = db.list_runs(limit=5)
if not recent_runs.empty:
    print(recent_runs)
else:
    print("   (No previous runs found)")

In [ ]:
# @title 🔍 Prerequisite Validation { display-mode: "form" }

print("="*70)
print("PREREQUISITE VALIDATION")
print("="*70)
print()

validation_passed = True
errors = []

# Check 1: Utils package exists
print("1/4 Checking utils package...")
if not os.path.exists('utils'):
    errors.append("Utils package not found. Run Section 1 setup cells first.")
    validation_passed = False
else:
    print("   ✅ Utils directory exists")

# Check 2: Critical modules present
print("2/4 Checking training modules...")
required_files = [
    'utils/training/task_spec.py',
    'utils/training/training_config.py',
    'utils/training/training_core.py',
]
missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    errors.append(f"Missing training modules: {', '.join(missing_files)}")
    validation_passed = False
else:
    print("   ✅ All training modules present")

# Check 3: Python can import utils
print("3/4 Testing imports...")
try:
    import sys
    if '.' not in sys.path:
        sys.path.insert(0, '.')
    
    from utils.training.task_spec import TaskSpec
    from utils.training.training_config import TrainingConfig
    print("   ✅ Core imports successful")
except ImportError as e:
    errors.append(f"Import test failed: {e}")
    validation_passed = False

# Check 4: Workspace configured
print("4/4 Checking workspace...")
if 'workspace_root' not in globals() or workspace_root is None:
    errors.append("Workspace not configured. Run Section 1 storage setup first.")
    validation_passed = False
else:
    print(f"   ✅ Workspace: {workspace_root}")

print()
if validation_passed:
    print("="*70)
    print("✅ ALL PREREQUISITES MET")
    print("="*70)
    print("\n✨ Ready to proceed with training setup")
else:
    print("="*70)
    print("❌ VALIDATION FAILED")
    print("="*70)
    print("\nErrors found:")
    for i, err in enumerate(errors, 1):
        print(f"   {i}. {err}")
    print("\n📝 Resolution steps:")
    print("   1. Restart runtime: Runtime → Restart runtime")
    print("   2. Run all cells in Section 1 (Setup & Drive Workspace) in order")
    print("   3. Wait for each cell to complete before proceeding")
    print("   4. Re-run this validation cell")
    raise RuntimeError("Prerequisites not met. See errors above.")


In [ ]:
# Import v3.5/v3.6 training infrastructure

# Ensure path is set
import sys
if '.' not in sys.path:
    sys.path.insert(0, '.')

print("📦 Loading training infrastructure (v3.5/v3.6)...")

try:
    # Core training components
    from utils.training.training_core import TrainingCoordinator
    from utils.training.training_config import TrainingConfig
    from utils.training.task_spec import TaskSpec
    from utils.training.metrics_tracker import MetricsTracker
    
    # v3.6 features
    from utils.training.drift_metrics import compute_dataset_profile, compare_profiles
    from utils.training.dashboard import TrainingDashboard
    
    # v3.5 features
    from utils.training.export_utilities import create_export_bundle
    
    # Model adapters
    from utils.adapters.model_adapter import UniversalModelAdapter, FlashAttentionWrapper
    
    # Data handling
    from utils.tokenization.data_module import SimpleDataModule
    
    print("✅ Training infrastructure loaded")
    print("   Core:")
    print("     - TrainingCoordinator (v3.5)")
    print("     - TrainingConfig (versioned configuration)")
    print("     - TaskSpec (modality-aware task definitions)")
    print("   v3.5 Features:")
    print("     - torch.compile integration (10-20% speedup)")
    print("     - VisionDataCollator (auto-selected for vision tasks)")
    print("     - Gradient accumulation tracking")
    print("     - Export bundle generation (production artifacts)")
    print("   v3.6 Features:")
    print("     - Distributed training guardrails (notebook safety)")
    print("     - Flash Attention support (2-4x attention speedup)")
    print("     - Drift visualization dashboard (10-panel layout)")
    print()
    print("🚀 Ready for professional training!")
    
except ImportError as e:
    print("❌ Failed to load training infrastructure!")
    print(f"\nError: {e}")
    print("\n" + "="*70)
    print("TROUBLESHOOTING")
    print("="*70)
    print("\n1. Ensure previous cells in Section 1 completed successfully")
    print("2. Check that utils package was downloaded (look for ✅ in output)")
    print("3. Try re-running the utils download cell")
    print("4. If problem persists, restart runtime and run from beginning")
    raise

<a id="section-2"></a>
# 📦 Section 2: Model Loading


Load your transformer model from Transformer Builder or use the example model.

**Options:**
- **Custom Model**: Provide Gist ID from Transformer Builder (auto-detected from URL)
- **Example Model**: GPT-2 style architecture for testing

**You will see:**
1. Model code preview
2. Architecture summary (layers, parameters, size)
3. GPU compatibility check


In [ ]:
# @title 🔗 Model Source Configuration { display-mode: "form" }

# Step 1: Try to extract from URL hash using JavaScript
from google.colab import output
import os
import json

# JavaScript to extract gist_id and model_name from URL hash
js_code = """
(function() {
    let gist_id = '';
    let model_name = '';

    try {
        // Try to read URL hash from parent window (Colab embedding)
        const hash = window.parent.location.hash || window.location.hash || '';

        if (hash) {
            // Parse hash parameters (e.g., #gist_id=abc123&name=MyModel)
            const params = new URLSearchParams(hash.substring(1));
            gist_id = params.get('gist_id') || '';
            model_name = params.get('name') || '';

            console.log('Extracted from URL hash:', {gist_id, model_name});
        }
    } catch (e) {
        console.log('Could not access URL hash:', e);
    }

    // Return as JSON string
    return JSON.stringify({gist_id: gist_id, model_name: model_name});
})();
"""

# Execute JavaScript and get returned values
try:
    url_params_json = output.eval_js(js_code)
    url_params = json.loads(url_params_json)
    gist_id_from_url = url_params.get('gist_id', '')
    model_name_from_url = url_params.get('model_name', '')
except Exception as e:
    print(f"⚠️  Could not extract from URL hash: {e}")
    gist_id_from_url = ''
    model_name_from_url = ''

# Step 2: Manual input forms (as fallback)
gist_id_manual = ""  #@param {type:"string"}
model_name_manual = "CustomTransformer"  #@param {type:"string"}

# Step 3: Environment variables (lowest priority)
gist_id_env = os.getenv('GIST_ID', '')
model_name_env = os.getenv('MODEL_NAME', '')

# Step 4: Determine final values (URL > Manual > Env)
gist_id = gist_id_from_url or gist_id_manual or gist_id_env
model_name = model_name_from_url or model_name_manual or model_name_env or 'CustomTransformer'

# Display source
print("="*60)
if gist_id:
    source = "URL hash" if gist_id_from_url else ("Manual input" if gist_id_manual else "Environment variable")
    print(f"✅ Model Source: {source}")
    print(f"   Gist ID: {gist_id}")
    print(f"   Model Name: {model_name}")
    print(f"\n   Loading custom model from Transformer Builder...")
else:
    print("ℹ️  No Gist ID provided")
    print("   Options to provide Gist ID:")
    print("   1. Open via Transformer Builder link (auto-detects from URL)")
    print("   2. Enter Gist ID in the form above")
    print("   3. Set GIST_ID environment variable")
    print("\n   Proceeding with example model for demonstration...")
print("="*60)


In [ ]:
# @title 📦 Load Model from Gist { display-mode: "form" }

import urllib.request
import json
import sys
import tempfile
import shutil

print("=" * 70)
print("MODEL LOADING")
print("=" * 70)
print()

# ==============================================================================
# VERIFY GIST ID WAS PROVIDED
# ==============================================================================

if 'gist_id' not in globals() or not gist_id:
    print("❌ ERROR: No Gist ID found!")
    print()
    print("==" * 35)
    print("🔙 GO BACK TO PREVIOUS CELL")
    print("==" * 35)
    print()
    print("You must run the Model Source Configuration cell first.")
    print()
    raise ValueError("Gist ID required - run previous cell first")

print(f"📥 Loading model from GitHub Gist: {gist_id}")
print()

# ==============================================================================
# FETCH GIST AND LOAD MODEL FILES - GitHub API Approach
# ==============================================================================

def _fetch_gist(gid: str) -> dict:
    """Fetch Gist data from GitHub API."""
    url = f"https://api.github.com/gists/{gid}"
    req = urllib.request.Request(url, headers={
        "Accept": "application/vnd.github+json",
        "User-Agent": "transformer-builder-colab"
    })
    try:
        with urllib.request.urlopen(req, timeout=20) as resp:
            return json.loads(resp.read().decode("utf-8"))
    except urllib.error.HTTPError as e:
        detail = f"HTTP {e.code}"
        try:
            body = e.read().decode("utf-8")
            if "rate limit" in body.lower():
                detail += " - GitHub API rate limit (try again in an hour)"
            elif e.code == 404:
                detail += " - Gist not found (check your Gist ID)"
        except:
            pass
        raise RuntimeError(f"GitHub API error: {detail}") from e
    except Exception as e:
        raise RuntimeError(f"Network error: {e}") from e

def _write(path: str, text: str):
    """Write text to file."""
    with open(path, "w") as f:
        f.write(text)

# Fetch Gist
try:
    gist_data = _fetch_gist(gist_id)
    files = gist_data.get("files") or {}

    # Check for required files
    if "model.py" not in files:
        raise RuntimeError("Gist is missing 'model.py' - please re-export from Transformer Builder")
    if "config.json" not in files:
        raise RuntimeError("Gist is missing 'config.json' - please re-export from Transformer Builder")

    model_code = files["model.py"].get("content", "")
    config_json = files["config.json"].get("content", "")

    if not model_code or not config_json:
        raise RuntimeError("Empty content in model.py or config.json")

    # Write to files
    _write("model.py", model_code)
    _write("config.json", config_json)

    print(f"✅ Model loaded successfully!")
    print(f"✅ Gist URL: {gist_data.get('html_url', 'N/A')}")
    print(f"✅ Model code: {len(model_code):,} bytes")
    print(f"✅ Config: {len(config_json):,} bytes")
    print()

    # Parse model name from config if available
    try:
        model_config = json.loads(config_json)
        if 'model_name' in model_config:
            model_name = model_config['model_name']
            print(f"✅ Model name: {model_name}")
        else:
            model_name = 'CustomTransformer'
            print(f"ℹ️  Using default name: {model_name}")
        print()
    except:
        model_name = 'CustomTransformer'
        print(f"⚠️  Could not parse config, using default name: {model_name}")

    # Store for next cell
    gist_loaded = True

except Exception as e:
    print(f"❌ Failed to load model from Gist!")
    print()
    print(f"Error: {e}")
    print()
    print("=" * 70)
    print("TROUBLESHOOTING")
    print("=" * 70)
    print()
    print("Common issues:")
    print("  1. Check your Gist ID is correct (go back to previous cell)")
    print("  2. Ensure you exported from Transformer Builder successfully")
    print("  3. Check you're not hitting GitHub rate limit (60 requests/hour)")
    print("  4. Try re-exporting from Transformer Builder")
    print()
    print("If the problem persists:")
    print(f"  • Gist URL: https://gist.github.com/{gist_id}")
    print("  • Verify the Gist contains model.py and config.json")
    print()

    # Fallback to example model
    print("⚠️  Falling back to example model for demonstration...")
    gist_loaded = False
    model_name = 'ExampleTransformer'

print("=" * 70)
print("✅ MODEL LOADING COMPLETE")
print("=" * 70)
print()
print("Model will be instantiated in the next cell.")
print()

# Display downloaded model code preview
if gist_loaded:
    print("\n📄 Model Code Preview:")
    print("=" * 60)
    with open('model.py', 'r') as f:
        model_lines = f.read().split('\n')
        # Show first 20 lines
        for i, line in enumerate(model_lines[:20], 1):
            print(f"{i:3d} | {line}")
        if len(model_lines) > 20:
            print(f"... ({len(model_lines) - 20} more lines)")
    print("=" * 60)

print(f"\n📊 Model: {model_name}")
if gist_loaded:
    print(f"   Config: {json.dumps(model_config, indent=2)}")


In [ ]:
# @title 🚀 Initialize Model { display-mode: "form" }

import torch
import torch.nn as nn
import inspect
from types import SimpleNamespace

# Detect device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Create model instance
if gist_loaded:
    # Custom model from Transformer Builder
    # Import the model from downloaded file
    try:
        sys.path.insert(0, '.')

        # Import all classes from model.py
        import importlib.util
        spec = importlib.util.spec_from_file_location("custom_model", "model.py")
        custom_model_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(custom_model_module)

        # Find the model class
        model_class = None
        for name, obj in vars(custom_model_module).items():
            if isinstance(obj, type) and issubclass(obj, nn.Module) and obj is not nn.Module:
                if name == model_name:
                    model_class = obj
                    break
        
        if model_class is None:
            # Fallback: find any nn.Module subclass
            for name, obj in vars(custom_model_module).items():
                if isinstance(obj, type) and issubclass(obj, nn.Module) and obj is not nn.Module:
                    model_class = obj
                    print(f"⚠️ Using {name} (expected {model_name})")
                    break
        
        if model_class:
            # Check constructor signature (KEY FIX from template.ipynb)
            sig = inspect.signature(model_class.__init__)
            params_list = [p for p in sig.parameters.values() if p.name != 'self']
            
            if len(params_list) == 0:
                # Parameterless constructor (Transformer Builder models)
                print("ℹ️  Model has parameterless constructor (Transformer Builder export)")
                model = model_class()
            else:
                # Parameterized constructor (traditional models)
                print(f"ℹ️  Model accepts {len(params_list)} parameter(s)")
                model = model_class(**model_config)
            
            print(f"✅ Custom model instantiated: {model.__class__.__name__}")
        else:
            raise Exception("No model class found in model.py")

    except Exception as e:
        print(f"❌ Failed to instantiate custom model: {e}")
        print("   Falling back to example model...")
        gist_loaded = False

if not gist_loaded:
    # Example model (fallback)
    print("📦 Loading example model (GPT-2 architecture)...")

    class ExampleTransformer(nn.Module):
        """Example GPT-2 style transformer for demonstration."""

        def __init__(self, vocab_size=50257, d_model=768, n_layers=12, n_heads=12, max_seq_len=1024):
            super().__init__()
            self.vocab_size = vocab_size
            self.d_model = d_model
            self.n_layers = n_layers
            self.n_heads = n_heads
            self.max_seq_len = max_seq_len

            self.embedding = nn.Embedding(vocab_size, d_model)
            self.position_embedding = nn.Embedding(max_seq_len, d_model)

            # Simple transformer layers
            self.layers = nn.ModuleList([
                nn.TransformerEncoderLayer(
                    d_model,
                    n_heads,
                    dim_feedforward=d_model*4,
                    batch_first=True,
                    dropout=0.1
                )
                for _ in range(n_layers)
            ])

            self.ln_f = nn.LayerNorm(d_model)
            self.lm_head = nn.Linear(d_model, vocab_size, bias=False)

        def forward(self, input_ids):
            batch_size, seq_len = input_ids.shape

            # Embeddings
            token_emb = self.embedding(input_ids)
            pos_ids = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
            pos_emb = self.position_embedding(pos_ids)

            x = token_emb + pos_emb

            # Transformer layers
            for layer in self.layers:
                x = layer(x)

            x = self.ln_f(x)
            logits = self.lm_head(x)

            return logits

    # Create example model
    model = ExampleTransformer()
    model_config = {
        'vocab_size': 50257,
        'd_model': 768,
        'n_layers': 12,
        'n_heads': 12,
        'max_seq_len': 1024
    }

    print(f"✅ Example model definition loaded")

# Move to device
model = model.to(device)

# Model summary
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n✅ Model initialized on {device}")
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")
print(f"   Model size: {total_params * 4 / 1e6:.1f} MB (fp32)")

# Create config object for training utilities
config_obj = SimpleNamespace(**model_config)
if not hasattr(config_obj, 'vocab_size'):
    config_obj.vocab_size = model_config.get('vocab_size', 50257)
if not hasattr(config_obj, 'max_seq_len'):
    config_obj.max_seq_len = model_config.get('max_seq_len', 1024)

print(f"\n🎯 Ready for training!")
print(f"\nℹ️  Note: Update Section 4 training config before starting training loop.")


<a id="section-3"></a>
# 📊 Section 3: Data Loading

Choose your data source (run ONE of the following cells):
- **Option 1**: HuggingFace Datasets (recommended)
- **Option 2**: Google Drive Upload
- **Option 3**: File Upload (small datasets)
- **Option 4**: Local Files (from previous sessions)
- **Option 5**: Synthetic Data (testing only)


In [ ]:
from datasets import load_dataset

# CONFIGURATION: Edit dataset name
dataset_name = "wikitext"  #@param {type:"string"}
config_name = "wikitext-2-raw-v1"  #@param {type:"string"}
max_samples = 1000  #@param {type:"integer"}

# Load dataset
dataset = load_dataset(dataset_name, config_name)
train_data = dataset['train'].select(range(min(max_samples, len(dataset['train']))))
val_data = dataset['validation'].select(range(min(100, len(dataset['validation']))))

print(f"✅ Loaded {len(train_data)} training samples, {len(val_data)} validation samples")
print(f"   Example: {train_data[0]}")

data_source = "huggingface"
dataset_info = {'name': dataset_name, 'config': config_name, 'train_size': len(train_data), 'val_size': len(val_data)}

In [ ]:
import os

drive_data_path = "/content/drive/MyDrive/TransformerTraining/datasets/my_data.txt"  #@param {type:"string"}

if os.path.exists(drive_data_path):
    with open(drive_data_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    split_idx = int(0.9 * len(lines))
    train_data = [line.strip() for line in lines[:split_idx]]
    val_data = [line.strip() for line in lines[split_idx:]]

    print(f"✅ Loaded {len(train_data)} training samples, {len(val_data)} validation samples")
    data_source = "google_drive"
    dataset_info = {'path': drive_data_path, 'train_size': len(train_data), 'val_size': len(val_data)}
else:
    print(f"❌ File not found: {drive_data_path}")
    print("   Please upload your data to Google Drive first")

In [ ]:
from google.colab import files
import io

# Upload file
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    content = uploaded[filename].decode('utf-8')
    lines = content.split('\n')

    split_idx = int(0.9 * len(lines))
    train_data = [line.strip() for line in lines[:split_idx]]
    val_data = [line.strip() for line in lines[split_idx:]]

    print(f"✅ Loaded {len(train_data)} training samples, {len(val_data)} validation samples")
    data_source = "file_upload"
    dataset_info = {'filename': filename, 'train_size': len(train_data), 'val_size': len(val_data)}

In [ ]:
import pickle
import os

cache_path = f'{workspace_root}/datasets/cached_data.pkl'

if os.path.exists(cache_path):
    with open(cache_path, 'rb') as f:
        data = pickle.load(f)

    train_data = data['train']
    val_data = data['val']

    print(f"✅ Loaded cached data: {len(train_data)} train, {len(val_data)} val")
    data_source = "cached"
    dataset_info = {'path': cache_path, 'train_size': len(train_data), 'val_size': len(val_data)}
else:
    print(f"❌ No cached data found at {cache_path}")
    print("   Run one of the other data loading options first")

In [ ]:
import torch

# Generate synthetic data for testing
vocab_size = 50257  # GPT-2 vocab
seq_len = 32
n_samples = 100

train_data = [torch.randint(0, vocab_size, (seq_len,)) for _ in range(n_samples)]
val_data = [torch.randint(0, vocab_size, (seq_len,)) for _ in range(20)]

print(f"✅ Generated {len(train_data)} synthetic training samples")
print(f"   ⚠️ Warning: Synthetic data is for testing only")
data_source = "synthetic"
dataset_info = {'vocab_size': vocab_size, 'seq_len': seq_len, 'train_size': len(train_data), 'val_size': len(val_data)}

In [ ]:
# @title 🎯 Task Specification (Auto-Detected) { display-mode: "form" }

# === SELF-CONTAINED IMPORTS ===
import sys
import os
if '.' not in sys.path:
    sys.path.insert(0, '.')

# Import with error handling
try:
    from utils.training.task_spec import TaskSpec
    from types import SimpleNamespace
    import torch
except ImportError as e:
    print("❌ Import failed!")
    print(f"\nError: {e}")
    print("\n" + "="*70)
    print("TROUBLESHOOTING")
    print("="*70)
    print("\n1. Run the 'Prerequisite Validation' cell above")
    print("2. Ensure Section 1 setup completed successfully")
    print("3. Check that utils package is downloaded")
    raise

print("=" * 70)
print("TASK SPECIFICATION")
print("=" * 70)
print()

# Auto-detect modality from data source
detected_modality = "text"  # Default
detected_task_type = "lm"   # Default

# Try to detect from data
if 'train_data' in globals() and train_data:
    try:
        sample = train_data[0] if isinstance(train_data, list) else train_data[0]
        
        # Check for vision data
        if isinstance(sample, dict):
            if 'pixel_values' in sample or 'image' in sample:
                detected_modality = "vision"
                detected_task_type = "vision_classification"
                print("🔍 Detected vision data (found pixel_values/image in sample)")
            elif 'input_ids' in sample or 'text' in sample:
                detected_modality = "text"
                detected_task_type = "lm"
                print("🔍 Detected text data (found input_ids/text in sample)")
        elif isinstance(sample, torch.Tensor):
            if sample.dim() >= 3:  # Likely image (C, H, W) or (B, C, H, W)
                detected_modality = "vision"
                detected_task_type = "vision_classification"
                print(f"🔍 Detected vision data (tensor shape: {sample.shape})")
            else:
                detected_modality = "text"
                detected_task_type = "lm"
                print(f"🔍 Detected text data (tensor shape: {sample.shape})")
    except Exception as e:
        print(f"⚠️  Could not auto-detect from data: {e}")
        print("   Using default: text/lm")

print(f"   Modality: {detected_modality}")
print(f"   Task type: {detected_task_type}")
print()

# Allow manual override
task_modality = detected_modality  #@param ["text", "vision", "audio", "tabular"]
task_type = detected_task_type  #@param ["lm", "classification", "seq2seq", "vision_classification", "vision_multilabel"]

print(f"📝 Final selection:")
print(f"   Modality: {task_modality}")
print(f"   Task type: {task_type}")
print()

# Create TaskSpec based on modality
if task_modality == "vision":
    # Vision task configuration
    image_size = getattr(config_obj, 'image_size', [3, 224, 224]) if 'config_obj' in globals() else [3, 224, 224]
    num_classes = getattr(config_obj, 'num_classes', 10) if 'config_obj' in globals() else 10
    
    task_spec = TaskSpec(
        name=f"{model_name}_vision" if 'model_name' in globals() else "vision_task",
        task_type=task_type,
        model_family="encoder_only",
        input_fields=["pixel_values"],
        target_field="labels",
        loss_type="cross_entropy",
        metrics=["loss", "accuracy"],
        modality="vision",
        input_schema={"image_size": image_size, "channels_first": True},
        output_schema={"num_classes": num_classes},
        preprocessing_config={
            "normalize": True,
            "mean": [0.485, 0.456, 0.406],  # ImageNet defaults
            "std": [0.229, 0.224, 0.225]
        }
    )
    
    print("✅ Vision TaskSpec created")
    print(f"   Image size: {image_size}")
    print(f"   Num classes: {num_classes}")
    print(f"   VisionDataCollator will be auto-selected (v3.5 feature)")
    
else:  # text modality
    # Text task configuration
    vocab_size = getattr(config_obj, 'vocab_size', 50257) if 'config_obj' in globals() else 50257
    max_seq_len = getattr(config_obj, 'max_seq_len', 128) if 'config_obj' in globals() else 128
    
    task_spec = TaskSpec(
        name=f"{model_name}_lm" if 'model_name' in globals() else "text_task",
        task_type=task_type,
        model_family="decoder_only",
        input_fields=["input_ids", "attention_mask"],
        target_field="labels",
        loss_type="cross_entropy",
        metrics=["loss", "perplexity", "accuracy"],
        modality="text",
        input_schema={"max_seq_len": max_seq_len, "vocab_size": vocab_size},
        output_schema={"vocab_size": vocab_size},
        special_tokens={"pad_token_id": 0},
        additional_config={"shift_labels": True}
    )
    
    print("✅ Text TaskSpec created")
    print(f"   Vocab size: {vocab_size}")
    print(f"   Max sequence length: {max_seq_len}")

print()
print("=" * 70)
print("✅ TASK SPECIFICATION COMPLETE")
print("=" * 70)
print()
print("💡 TaskSpec enables:")
print("   - Modality-aware data collation")
print("   - Drift detection (v3.6)")
print("   - Export bundle generation (v3.5)")
print()

In [ ]:
# @title 📝 Data Tokenization & Preprocessing { display-mode: "form" }

# === SELF-CONTAINED IMPORTS ===
import sys
import os
if '.' not in sys.path:
    sys.path.insert(0, '.')

try:
    import torch
    from utils.tokenization.adaptive_tokenizer import AdaptiveTokenizer
    from utils.tokenization.data_collator import LanguageModelingDataCollator
    import time
except ImportError as e:
    print("❌ Import failed!")
    print(f"\nError: {e}")
    print("\nTroubleshooting:")
    print("1. Ensure utils package downloaded (Section 1)")
    print("2. Re-run notebook from start")
    raise

# === PREREQUISITE CHECKS ===
if 'train_data' not in globals() or 'val_data' not in globals():
    raise NameError("train_data/val_data not found. Run data loading cell first.")
if 'config_obj' not in globals():
    raise NameError("config_obj not found. Run model loading cell first.")
if 'task_spec' not in globals():
    raise NameError("task_spec not found. Run task specification cell first.")

print("="*70)
print("DATA TOKENIZATION & PREPROCESSING")
print("="*70)
print()

# === MODALITY-AWARE PROCESSING ===
modality = task_spec.modality
print(f"📋 Modality: {modality}")
print(f"📦 Vocab Size: {config_obj.vocab_size}")
print(f"📏 Max Sequence Length: {task_spec.max_seq_len}")
print()

if modality == 'text':
    # === TEXT TOKENIZATION ===
    
    # Step 1: Auto-detect text field
    text_fields = ['text', 'content', 'sentence', 'document', 'body', 'input', 'prompt']
    text_field = next((f for f in text_fields if f in train_data.features), None)
    
    if text_field is None:
        print("❌ No text field found in dataset!")
        print(f"   Available fields: {list(train_data.features.keys())}")
        raise ValueError("Cannot auto-detect text field. Please specify manually.")
    
    print(f"✅ Text field detected: '{text_field}'")
    print()
    
    # Step 2: Load or create tokenizer via AdaptiveTokenizer
    print("🔧 Loading tokenizer...")
    tokenizer = AdaptiveTokenizer.load_or_create(
        vocab_size=config_obj.vocab_size,
        dataset=train_data
    )
    print(f"✅ Tokenizer loaded: {tokenizer.__class__.__name__}")
    
    # Get vocab size safely
    if hasattr(tokenizer, 'vocab_size'):
        tok_vocab = tokenizer.vocab_size
    elif hasattr(tokenizer, 'get_vocab_size'):
        tok_vocab = tokenizer.get_vocab_size()
    else:
        tok_vocab = len(tokenizer.get_vocab()) if hasattr(tokenizer, 'get_vocab') else 'unknown'
    print(f"   Vocab size: {tok_vocab}")
    print()
    
    # Step 3: Define tokenization function (lazy evaluation)
    def tokenize_batch(batch):
        """Tokenize a batch of examples using lazy evaluation."""
        # Handle different tokenizer interfaces
        if hasattr(tokenizer, '__call__'):
            # HuggingFace tokenizer
            result = tokenizer(
                batch[text_field],
                truncation=True,
                max_length=task_spec.max_seq_len,
                padding=False,  # Defer to collator for dynamic padding
                return_tensors=None  # Keep as lists for now
            )
        elif hasattr(tokenizer, 'encode'):
            # Simple encode interface
            result = {
                'input_ids': [tokenizer.encode(text, max_length=task_spec.max_seq_len, truncation=True) 
                             for text in batch[text_field]]
            }
        else:
            raise ValueError(f"Tokenizer {type(tokenizer)} has no compatible encode method")
        return result
    
    # Step 4: Apply tokenization via dataset.map (lazy loading)
    print("⚙️  Tokenizing datasets (this may take a moment)...")
    start_time = time.time()
    
    # Determine cache behavior
    overwrite_cache = False  # Set to True during debugging
    num_proc = min(4, os.cpu_count() or 1)  # Parallel processing
    
    # Tokenize train dataset
    train_data = train_data.map(
        tokenize_batch,
        batched=True,
        remove_columns=[text_field],  # Remove raw text to save memory
        num_proc=num_proc if os.name != 'nt' else 1,  # Windows compatibility
        load_from_cache_file=not overwrite_cache,
        desc="Tokenizing train data"
    )
    
    # Tokenize validation dataset
    val_data = val_data.map(
        tokenize_batch,
        batched=True,
        remove_columns=[text_field],
        num_proc=num_proc if os.name != 'nt' else 1,
        load_from_cache_file=not overwrite_cache,
        desc="Tokenizing val data"
    )
    
    elapsed = time.time() - start_time
    print(f"✅ Tokenization complete in {elapsed:.2f}s")
    print()
    
    # Step 5: Set format to PyTorch (lazy, zero-copy)
    train_data.set_format(type='torch', columns=['input_ids'])
    val_data.set_format(type='torch', columns=['input_ids'])
    
    # Step 6: Create collator for dynamic padding
    data_collator = LanguageModelingDataCollator(
        tokenizer=tokenizer,
        mlm=False,  # Causal LM (GPT-style)
        padding_side='right'
    )
    
    # Step 7: Display statistics
    print("📊 Tokenization Statistics:")
    print(f"   Train samples: {len(train_data)}")
    print(f"   Val samples: {len(val_data)}")
    
    # Compute length statistics
    sample_size = min(1000, len(train_data))
    train_lengths = [len(ex['input_ids']) for ex in train_data.select(range(sample_size))]
    mean_len = sum(train_lengths) / len(train_lengths)
    max_len = max(train_lengths)
    truncated_pct = sum(1 for l in train_lengths if l == task_spec.max_seq_len) / len(train_lengths) * 100
    
    print(f"   Mean length: {mean_len:.1f} tokens")
    print(f"   Max length: {max_len} tokens")
    print(f"   Truncated: {truncated_pct:.1f}%")
    
    if truncated_pct > 10:
        print(f"   ⚠️  Warning: >{truncated_pct:.0f}% sequences truncated. Consider increasing max_seq_len.")
    
    # Step 8: Show example
    print()
    print("🔍 Example tokenized sample:")
    example = train_data[0]
    print(f"   Input IDs shape: {example['input_ids'].shape}")
    print(f"   First 20 tokens: {example['input_ids'][:20].tolist()}")
    if hasattr(tokenizer, 'decode'):
        try:
            decoded = tokenizer.decode(example['input_ids'][:50])
            print(f"   Decoded preview: {decoded[:100]}...")
        except:
            print(f"   (decode not available for this tokenizer)")

elif modality == 'vision':
    # === VISION PREPROCESSING ===
    try:
        from torchvision import transforms
        from utils.tokenization.data_collator import VisionDataCollator
        
        print("🖼️  Vision modality detected")
        
        # Get image size from task_spec or config
        image_size = getattr(task_spec, 'image_size', 224)
        
        # Define image transforms
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        
        def preprocess_images(batch):
            """Preprocess images."""
            images = [transform(img.convert('RGB')) for img in batch['image']]
            return {'pixel_values': images}
        
        print("⚙️  Preprocessing images...")
        train_data = train_data.map(
            preprocess_images,
            batched=True,
            remove_columns=['image'],
            desc="Preprocessing train images"
        )
        val_data = val_data.map(
            preprocess_images,
            batched=True,
            remove_columns=['image'],
            desc="Preprocessing val images"
        )
        
        train_data.set_format(type='torch', columns=['pixel_values', 'label'])
        val_data.set_format(type='torch', columns=['pixel_values', 'label'])
        
        data_collator = VisionDataCollator(image_size)
        print(f"✅ Vision preprocessing complete")
    except ImportError as e:
        print(f"⚠️  Vision preprocessing failed: {e}")
        print("   Install torchvision: !pip install torchvision")
        data_collator = None

elif modality == 'audio':
    # === AUDIO PREPROCESSING ===
    print("🎵 Audio modality detected")
    print("⚠️  Audio preprocessing not yet implemented")
    print("   Falling back to default behavior")
    data_collator = None

else:
    # === UNKNOWN MODALITY ===
    print(f"⚠️  Unknown modality: {modality}")
    print("   Proceeding without preprocessing")
    data_collator = None

print()
print("="*70)
print("✅ PREPROCESSING COMPLETE")
print("="*70)
print()
print("📦 Output format:")
print(f"   train_data: {type(train_data)}")
print(f"   val_data: {type(val_data)}")
print(f"   data_collator: {type(data_collator).__name__ if data_collator else 'None'}")
print()
print("✨ Ready for training!")


<a id="section-4"></a>
# ⚙️ Section 4: Training Configuration

Configure hyperparameters using Colab forms below.

In [ ]:
# @title ⚙️ Training Configuration (v3.5/v3.6 Features) { display-mode: "form" }

# === SELF-CONTAINED IMPORTS ===
import sys
import os
if '.' not in sys.path:
    sys.path.insert(0, '.')

# Import with error handling
try:
    from utils.training.training_config import TrainingConfig
    import torch
except ImportError as e:
    print("❌ Import failed!")
    print(f"\nError: {e}")
    print("\n" + "="*70)
    print("TROUBLESHOOTING")
    print("="*70)
    print("\n1. Run the 'Prerequisite Validation' cell in Section 1")
    print("2. Ensure utils package is properly installed")
    print("3. Run all previous cells in order")
    raise

# Prerequisite check
if 'workspace_root' not in globals():
    print("❌ workspace_root not found!")
    print("\n📍 You are here: Section 4 (Training Configuration)")
    print("\n⚠️  Required: Run Section 1 (Setup & Drive Workspace) first")
    raise NameError("workspace_root not defined. Run Section 1 setup cells.")

if 'task_spec' not in globals():
    print("❌ task_spec not found!")
    print("\n📍 You are here: Section 4 (Training Configuration)")
    print("\n⚠️  Required: Run Section 3 (Task Specification) first")
    raise NameError("task_spec not defined. Run Task Specification cell in Section 3.")

print("=" * 70)
print("TRAINING CONFIGURATION")
print("=" * 70)
print()

# === HYPERPARAMETERS ===
learning_rate = 5e-5  #@param {type:"number"}
batch_size = 4  #@param {type:"integer"}
epochs = 10  #@param {type:"integer"}
warmup_ratio = 0.1  #@param {type:"number"}
weight_decay = 0.01  #@param {type:"number"}
gradient_clip_norm = 1.0  #@param {type:"number"}

# === v3.5 PERFORMANCE FEATURES ===
# torch.compile: 10-20% speedup (PyTorch 2.0+)
compile_mode = "default"  #@param ["None", "default", "reduce-overhead", "max-autotune"]
compile_mode = None if compile_mode == "None" else compile_mode

# Gradient accumulation: effective batch size = batch_size * accumulation_steps
gradient_accumulation_steps = 1  #@param {type:"integer"}

# === v3.5 EXPORT FEATURES ===
# Export bundle generation
export_bundle = True  #@param {type:"boolean"}
export_formats_str = "onnx,torchscript,pytorch"  #@param {type:"string"}
export_formats = [fmt.strip() for fmt in export_formats_str.split(',')]

# === STANDARD TRAINING FEATURES ===
use_amp = True  #@param {type:"boolean"}
deterministic = False  #@param {type:"boolean"}
run_name = "training-run"  #@param {type:"string"}
random_seed = 42  #@param {type:"integer"}

# Create TrainingConfig
training_config = TrainingConfig(
    # Hyperparameters
    learning_rate=learning_rate,
    batch_size=batch_size,
    epochs=epochs,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    max_grad_norm=gradient_clip_norm,
    
    # v3.5 features
    compile_mode=compile_mode,
    gradient_accumulation_steps=gradient_accumulation_steps,
    export_bundle=export_bundle,
    export_formats=export_formats,
    export_dir=f'{workspace_root}/exports',
    
    # Standard features
    use_amp=use_amp,
    deterministic=deterministic,
    random_seed=random_seed,
    run_name=run_name,
    
    # Directories
    checkpoint_dir=f'{workspace_root}/checkpoints',
)

# Validate configuration
training_config.validate()

# Save to Drive with proper filename
import os
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
config_filename = f"config_{training_config.run_name}_{timestamp}.json"
config_path = os.path.join(f'{workspace_root}/configs', config_filename)
training_config.save(config_path)
print(f"✅ Config saved: {config_path}")
print()

# Display configuration summary
print("HYPERPARAMETERS")
print("-" * 70)
print(f"{'Run Name:':<30} {training_config.run_name}")
print(f"{'Learning Rate:':<30} {training_config.learning_rate}")
print(f"{'Batch Size (per step):':<30} {training_config.batch_size}")
print(f"{'Effective Batch Size:':<30} {training_config.batch_size * training_config.gradient_accumulation_steps}")
print(f"{'Epochs:':<30} {training_config.epochs}")
print(f"{'Warmup Ratio:':<30} {training_config.warmup_ratio}")
print(f"{'Weight Decay:':<30} {training_config.weight_decay}")
print(f"{'Gradient Clipping:':<30} {gradient_clip_norm}")
print()

print("v3.5 PERFORMANCE FEATURES")
print("-" * 70)
print(f"{'torch.compile:':<30} {compile_mode or 'Disabled'}")
if compile_mode:
    expected_speedup = {"default": "10-15%", "reduce-overhead": "15-20%", "max-autotune": "20-30%"}
    print(f"{'  Expected speedup:':<30} ~{expected_speedup.get(compile_mode, '10-20%')}")
    print(f"{'  Compilation time:':<30} {'~30s' if compile_mode == 'max-autotune' else '~5s'}")
print()
print(f"{'Gradient Accumulation:':<30} {gradient_accumulation_steps}x")
if gradient_accumulation_steps > 1:
    log_reduction = 100 * (1 - 1/gradient_accumulation_steps)
    print(f"{'  W&B log reduction:':<30} ~{log_reduction:.0f}%")
    print(f"{'  Memory efficient:':<30} Train larger models on same GPU")
print()

print("v3.5 EXPORT FEATURES")
print("-" * 70)
print(f"{'Export Bundle:':<30} {'Enabled' if export_bundle else 'Disabled'}")
if export_bundle:
    print(f"{'  Formats:':<30} {', '.join(export_formats)}")
    print(f"{'  Output dir:':<30} {training_config.export_dir}")
    print(f"{'  Includes:':<30} Dockerfile, inference.py, README")
print()

print("v3.6 AUTOMATIC FEATURES")
print("-" * 70)
print(f"{'Distributed Guardrails:':<30} ✅ Active (notebook detection)")
print(f"{'  Prevents:':<30} DDP/FSDP zombie processes in Jupyter/Colab")
print()
print(f"{'Flash Attention:':<30} Auto-enabled (PyTorch 2.0+ + CUDA)")
if torch.__version__ >= "2.0.0" and torch.cuda.is_available():
    print(f"{'  Status:':<30} ✅ Will be enabled during training")
    print(f"{'  Expected speedup:':<30} 2-4x for attention operations")
elif torch.__version__ < "2.0.0":
    print(f"{'  Status:':<30} ⚠️  Requires PyTorch 2.0+")
else:
    print(f"{'  Status:':<30} ⚠️  Requires CUDA (GPU)")
print()
print(f"{'Drift Visualization:':<30} Available via plot_with_drift()")
print(f"{'  Panels:':<30} 10-panel dashboard (6 training + 4 drift)")
print()

print("STANDARD FEATURES")
print("-" * 70)
print(f"{'Mixed Precision (AMP):':<30} {'Enabled' if use_amp else 'Disabled'}")
print(f"{'Deterministic Mode:':<30} {'Enabled' if deterministic else 'Disabled (fast)'}")
print(f"{'Random Seed:':<30} {random_seed}")
print(f"{'Data Source:':<30} {data_source if 'data_source' in globals() else 'N/A'}")
print()

print("=" * 70)
print("✅ CONFIGURATION COMPLETE")
print("=" * 70)
print()
print("💡 Proceed to Section 6 for training with all features enabled")

In [ ]:
# Display configuration summary
print("=" * 60)
print(" " * 15 + "TRAINING CONFIGURATION")
print("=" * 60)
print(f"{'Run Name:':<25} {training_config.run_name}")
print(f"{'Learning Rate:':<25} {training_config.learning_rate}")
print(f"{'Batch Size (effective):':<25} {training_config.batch_size * training_config.gradient_accumulation_steps}")
print(f"{'Epochs:':<25} {training_config.epochs}")
print(f"{'Warmup Ratio:':<25} {training_config.warmup_ratio}")
print(f"{'Gradient Clipping:':<25} {training_config.max_grad_norm}")
print(f"{'AMP Enabled:':<25} {training_config.use_amp}")
print(f"{'Deterministic:':<25} {training_config.deterministic}")
print(f"{'Random Seed:':<25} {training_config.random_seed}")
print(f"{'Data Source:':<25} {data_source if 'data_source' in globals() else 'N/A'}")
print("=" * 60)

### Training Mode Selection

Based on your `epochs` setting:
- **epochs <= 5**: ⚡ Fast Mode (~5 min)
- **epochs <= 15**: ⚖️ Balanced Mode (~15 min)
- **epochs > 15**: 💎 Quality Mode (45+ min)

Proceed to training in Section 5 ⬇️

<a id="section-5"></a>
# 🔬 Section 5: W&B Tracking Setup (Optional)

Enable Weights & Biases for cloud-based experiment tracking.

In [ ]:
import wandb
from getpass import getpass

use_wandb = True  #@param {type:"boolean"}
wandb_project = "transformer-training"  #@param {type:"string"}
wandb_entity = ""  #@param {type:"string"}

if use_wandb:
    # Login to W&B
    wandb_key = getpass("Enter W&B API key (or leave blank to skip): ")
    if wandb_key:
        wandb.login(key=wandb_key)

        # Initialize run
        wandb.init(
            project=wandb_project,
            entity=wandb_entity if wandb_entity else None,
            name=config.run_name,
            config=config.to_dict(),
            tags=[data_source, f"epochs_{epochs}"]
        )
        print(f"✅ W&B initialized: {wandb.run.url}")
    else:
        use_wandb = False
        print("⚠️ W&B skipped - training will use local tracking only")
else:
    print("ℹ️ W&B disabled - using local SQLite tracking")

In [ ]:
# @title 📊 Dataset Drift Profiling (v3.6 - Optional, 1-2 min) { display-mode: "form" }

enable_drift_detection = True  #@param {type:"boolean"}
drift_sample_size = 1000  #@param {type:"integer"}

drift_data = None

if enable_drift_detection:
    print("=" * 70)
    print("DATASET DRIFT PROFILING (v3.6)")
    print("=" * 70)
    print()
    print("📊 Computing dataset profiles for drift detection...")
    print(f"   Sample size: {min(drift_sample_size, len(train_data))} from training")
    print(f"   Sample size: {min(drift_sample_size, len(val_data))} from validation")
    print()
    
    try:
        # Profile training dataset (reference)
        print("1/2 Profiling training dataset...")
        train_subset = train_data[:drift_sample_size] if isinstance(train_data, list) else train_data
        train_profile = compute_dataset_profile(
            dataset=train_subset,
            task_spec=task_spec,
            sample_size=drift_sample_size
        )
        print("   ✅ Training profile complete")
        
        # Profile validation dataset (comparison)
        print("2/2 Profiling validation dataset...")
        val_subset = val_data[:drift_sample_size] if isinstance(val_data, list) else val_data
        val_profile = compute_dataset_profile(
            dataset=val_subset,
            task_spec=task_spec,
            sample_size=drift_sample_size
        )
        print("   ✅ Validation profile complete")
        print()
        
        # Compare profiles
        print("🔍 Computing drift scores...")
        drift_comparison = compare_profiles(train_profile, val_profile)
        
        # Display results
        print()
        print("DRIFT DETECTION RESULTS")
        print("-" * 70)
        
        max_drift = 0.0
        for metric, score in drift_comparison['drift_scores'].items():
            max_drift = max(max_drift, score)
            
            # Determine status
            if score < 0.1:
                status_emoji = '✅'
                status_text = 'Healthy'
            elif score < 0.2:
                status_emoji = '⚠️ '
                status_text = 'Warning'
            else:
                status_emoji = '🚨'
                status_text = 'Critical'
            
            print(f"{status_emoji} {metric:<25} {score:.4f} ({status_text})")
        
        print("-" * 70)
        print()
        
        # Overall status
        if max_drift < 0.1:
            overall_status = 'ok'
            print("✅ HEALTHY: Minimal drift detected")
            print("   Training and validation datasets are well-matched")
        elif max_drift < 0.2:
            overall_status = 'warn'
            print("⚠️  WARNING: Moderate drift detected")
            print("   Recommendation:")
            print("   - Monitor training metrics closely")
            print("   - Consider reviewing train/val split")
        else:
            overall_status = 'alert'
            print("🚨 CRITICAL: Significant drift detected!")
            print("   Action Required:")
            print("   - Check data preprocessing consistency")
            print("   - Review train/val split strategy")
            print("   - Investigate distribution shift causes")
            
            if task_modality == "text":
                print()
                print("   Common causes for text:")
                print("   - Different text sources (news vs social media)")
                print("   - Language/domain shift (formal vs casual)")
                print("   - Tokenization inconsistencies")
            elif task_modality == "vision":
                print()
                print("   Common causes for vision:")
                print("   - Different image preprocessing")
                print("   - Lighting conditions (indoor vs outdoor)")
                print("   - Camera/sensor differences")
        
        print()
        print(f"Max Drift Score: {max_drift:.4f}")
        print()
        
        # Package for dashboard visualization
        drift_data = {
            'ref_profile': train_profile,
            'new_profile': val_profile,
            'drift_scores': drift_comparison['drift_scores'],
            'status': overall_status
        }
        
        print("=" * 70)
        print("✅ DRIFT PROFILING COMPLETE")
        print("=" * 70)
        print()
        print("💡 Drift visualization will be available in Section 7 dashboard")
        print()
        
    except Exception as e:
        print(f"❌ Drift profiling failed: {e}")
        print("   Continuing without drift detection")
        drift_data = None
else:
    print("=" * 70)
    print("DRIFT PROFILING SKIPPED")
    print("=" * 70)
    print()
    print("ℹ️  Drift detection disabled")
    print("   Set enable_drift_detection=True to enable")
    print()
    print("💡 Benefits of drift detection:")
    print("   - Early warning of data distribution shifts")
    print("   - Helps diagnose training issues")
    print("   - Validates train/val split quality")
    print("=" * 70)
    print()

<a id="section-6"></a>
# 🏋️ Section 6: Training Loop

Main training loop with live visualization and checkpointing.

In [ ]:
# @title 🚀 TrainingCoordinator Setup (v3.5/v3.6) { display-mode: "form" }

from utils.training.seed_manager import set_random_seed

print("=" * 70)
print("TRAINING COORDINATOR SETUP")
print("=" * 70)
print()

# Set random seed for reproducibility
print(f"🎲 Setting random seed: {training_config.random_seed}")
set_random_seed(training_config.random_seed, training_config.deterministic)
print(f"   Mode: {'Deterministic (bit-exact)' if training_config.deterministic else 'Fast (cuDNN auto-tuning)'}")
print()

# Create data module
print("📦 Creating SimpleDataModule...")

# For text tasks, we need to handle tokenization
if task_spec.modality == "text":
    # Simple tokenizer setup for demonstration
    # In production, you'd use a proper tokenizer
    print("   Modality: TEXT")
    print("   Note: Using simplified tokenization for demonstration")
    print("   For production, implement proper tokenization in data loading")
    
    # Create simple wrapper if data is already tokenized
    final_train_data = train_data
    final_val_data = val_data
    tokenizer = None
    
else:  # vision
    print(f"   Modality: VISION")
    print(f"   VisionDataCollator will be auto-selected (v3.5)")
    final_train_data = train_data
    final_val_data = val_data
    tokenizer = None

# Create data module
data_module = SimpleDataModule(
    train_dataset=final_train_data,
    val_dataset=final_val_data,
    task_spec=task_spec,
    batch_size=training_config.batch_size,
    num_workers=2,
    tokenizer=tokenizer
)

print(f"✅ Data module created")
approx_train_batches = len(final_train_data) // training_config.batch_size
approx_val_batches = len(final_val_data) // training_config.batch_size
print(f"   Train batches: ~{approx_train_batches}")
print(f"   Val batches: ~{approx_val_batches}")
print()

# Initialize TrainingCoordinator
print("🏗️  Initializing TrainingCoordinator...")

# Determine precision string
if training_config.use_amp:
    precision_str = '16-mixed'  # PyTorch Lightning 2.x format
else:
    precision_str = '32-true'

coordinator = TrainingCoordinator(
    output_dir=training_config.checkpoint_dir,
    use_gpu=torch.cuda.is_available(),
    precision=precision_str,
    gradient_clip_val=training_config.max_grad_norm,
    strategy='auto',  # v3.6: Notebook guardrails will auto-adjust
    devices='auto'
)

print(f"✅ Coordinator initialized")
print(f"   Output dir: {training_config.checkpoint_dir}")
print(f"   Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print(f"   Precision: {precision_str}")
print(f"   Strategy: auto (notebook-safe)")
print(f"   Gradient clipping: {training_config.max_grad_norm}")
print()

# Display v3.6 automatic features
print("v3.6 AUTOMATIC FEATURES")
print("-" * 70)

# Check Flash Attention
if torch.cuda.is_available() and torch.__version__ >= "2.0.0":
    # Flash Attention will be detected by UniversalModelAdapter
    print("Flash Attention:")
    print(f"   ✅ Available (PyTorch {torch.__version__} + CUDA)")
    print(f"   Will be auto-enabled during training")
    print(f"   Expected speedup: 2-4x for attention operations")
else:
    print("Flash Attention:")
    if not torch.cuda.is_available():
        print(f"   ⚠️  Requires CUDA (currently on CPU)")
    else:
        print(f"   ⚠️  Requires PyTorch 2.0+ (currently {torch.__version__})")
    print(f"   Falling back to standard attention")

print()

# Distributed guardrails
print("Distributed Training Guardrails:")
print(f"   ✅ Active (notebook environment detection)")
print(f"   Prevents DDP/FSDP zombie processes in Jupyter/Colab")

print()

# Display v3.5 features
print("v3.5 FEATURES")
print("-" * 70)

if training_config.compile_mode:
    print(f"torch.compile:")
    print(f"   ✅ Enabled ({training_config.compile_mode} mode)")
    print(f"   Expected speedup: ~10-20%")
    print(f"   First epoch will be slower due to compilation")
else:
    print(f"torch.compile: Disabled")

print()

if training_config.gradient_accumulation_steps > 1:
    print(f"Gradient Accumulation:")
    print(f"   ✅ {training_config.gradient_accumulation_steps}x steps")
    print(f"   Effective batch size: {training_config.batch_size * training_config.gradient_accumulation_steps}")
    log_reduction = 100 * (1 - 1/training_config.gradient_accumulation_steps)
    print(f"   W&B log reduction: ~{log_reduction:.0f}%")

print()
print("=" * 70)
print("✅ SETUP COMPLETE - Ready for Training")
print("=" * 70)
print()
print("💡 Run the next cell to start training")

In [ ]:
# @title ▶️ Run Training (Professional Pipeline) { display-mode: "form" }

import time

print("=" * 70)
print("TRAINING EXECUTION")
print("=" * 70)
print()

start_time = time.time()

print(f"🏋️  Starting training with {training_config.epochs} epochs...")
print(f"   Run name: {training_config.run_name}")
print(f"   Batch size: {training_config.batch_size}")
print(f"   Learning rate: {training_config.learning_rate}")
print()

# Note: The actual train() method signature depends on your TrainingCoordinator implementation
# This is a simplified version - adjust parameters based on actual implementation

try:
    # Execute training
    # Simplified call - adjust based on actual TrainingCoordinator.train() signature
    print("⚙️  Training in progress...")
    print("   (This may take several minutes depending on your configuration)")
    print()
    
    # For this notebook refactor, we'll use a simplified training approach
    # that leverages the existing utils while maintaining notebook compatibility
    
    from utils.tier3_training_utilities import test_fine_tuning
    
    # Run training using tier3 utilities
    # Note: train_data and val_data are now properly tokenized from the preprocessing cell
    results = test_fine_tuning(
        model=model,
        config=config_obj,
        train_data=train_data,  # ✅ Pass tokenized data from preprocessing cell
        val_data=val_data,      # ✅ Pass tokenized data from preprocessing cell
        n_epochs=training_config.epochs,
        learning_rate=training_config.learning_rate,
        batch_size=training_config.batch_size,
        random_seed=training_config.random_seed,
        deterministic=training_config.deterministic,
        use_wandb=use_wandb if 'use_wandb' in globals() else False,
        gradient_accumulation_steps=training_config.gradient_accumulation_steps,
        weight_decay=training_config.weight_decay,
        use_amp=training_config.use_amp
    )
    
    training_duration = time.time() - start_time
    
    print()
    print("=" * 70)
    print("✅ TRAINING COMPLETED SUCCESSFULLY!")
    print("=" * 70)
    print()
    print(f"Training Duration: {training_duration/60:.1f} minutes")
    print()
    
    # Display results summary
    print("FINAL RESULTS")
    print("-" * 70)
    print(f"{'Final Training Loss:':<30} {results['final_loss']:.4f}")
    
    if 'metrics_summary' in results:
        metrics_df = results['metrics_summary']
        last_epoch = metrics_df.iloc[-1]
        
        if 'val/loss' in last_epoch:
            print(f"{'Final Validation Loss:':<30} {last_epoch['val/loss']:.4f}")
        if 'val/perplexity' in last_epoch:
            print(f"{'Final Perplexity:':<30} {last_epoch['val/perplexity']:.2f}")
        if 'val/accuracy' in last_epoch:
            print(f"{'Final Accuracy:':<30} {last_epoch['val/accuracy']*100:.2f}%")
    
    print()
    
    # Best epoch info
    if 'best_epoch' in results:
        print(f"Best Epoch: {results['best_epoch'] + 1}")
        print(f"Best Validation Loss: {results.get('best_val_loss', 'N/A')}")
    
    print()
    
    # Save to experiment database
    if 'db' in globals():
        try:
            run_id = db.log_run(
                run_name=training_config.run_name,
                config=training_config.to_dict(),
                notes=f"Trained with v3.5/v3.6 features | {data_source} data"
            )
            
            # Log final metrics
            if 'metrics_summary' in results:
                final_metrics = results['metrics_summary'].iloc[-1].to_dict()
                for key, value in final_metrics.items():
                    if isinstance(value, (int, float)):
                        db.log_metric(run_id, key, value, epoch=training_config.epochs-1)
            
            db.update_run_status(run_id, 'completed')
            print(f"✅ Run logged to ExperimentDB (ID: {run_id})")
        except Exception as e:
            print(f"⚠️  Could not log to ExperimentDB: {e}")
    
    print()
    print("=" * 70)
    print("NEXT STEPS")
    print("=" * 70)
    print()
    print("1. 📊 View Results: Run Section 7 for comprehensive dashboard")
    print("2. 💾 Download: Use Section 8 to download checkpoints")
    if training_config.export_bundle:
        print("3. 📦 Export Bundle: Available in Section 8")
    print()
    
except Exception as e:
    print()
    print("=" * 70)
    print("❌ TRAINING FAILED")
    print("=" * 70)
    print()
    print(f"Error: {e}")
    print()
    print("Troubleshooting:")
    print("  - Check that model and data are compatible")
    print("  - Verify all previous cells ran successfully")
    print("  - Try reducing batch size if out of memory")
    print("  - Check logs above for specific error details")
    print()
    raise

<a id="section-7"></a>
# 📈 Section 7: Analysis & Visualization

Analyze training results with comprehensive dashboards.

In [ ]:
# @title 📈 Training Dashboard (v3.6: With Drift Visualization) { display-mode: "form" }

print("=" * 70)
print("TRAINING DASHBOARD GENERATION")
print("=" * 70)
print()

# Extract metrics from results
if 'results' in globals() and 'metrics_summary' in results:
    metrics_df = results['metrics_summary']
    print(f"✅ Metrics loaded: {len(metrics_df)} epochs")
else:
    print("⚠️  No training results found")
    print("   Please run Section 6 training first")
    metrics_df = None

if metrics_df is not None:
    # Create dashboard
    print()
    print("📊 Creating dashboard...")
    
    # Determine dashboard type
    if drift_data is not None:
        print("   Type: Enhanced (10-panel with drift visualization)")
        dashboard_size = (20, 14)  # Larger for drift panels
    else:
        print("   Type: Standard (6-panel)")
        dashboard_size = (18, 12)
    
    dashboard = TrainingDashboard(figsize=dashboard_size)
    
    try:
        if drift_data is not None:
            # v3.6: Enhanced dashboard with drift visualization
            print("   Building 10-panel layout...")
            print("     - 6 training metric panels")
            print("     - 4 drift detection panels")
            
            fig = dashboard.plot_with_drift(
                metrics_df=metrics_df,
                drift_data=drift_data,
                config=training_config,
                title=f"Training Dashboard + Drift Analysis: {training_config.run_name}"
            )
            
            print("   ✅ Enhanced dashboard created (10 panels)")
            
        else:
            # Standard 6-panel dashboard
            print("   Building 6-panel layout...")
            
            fig = dashboard.plot(
                metrics_df=metrics_df,
                config=training_config,
                title=f"Training Dashboard: {training_config.run_name}"
            )
            
            print("   ✅ Standard dashboard created (6 panels)")
        
        # Save to Drive
        dashboard_path = f'{workspace_root}/results/{training_config.run_name}_dashboard.png'
        dashboard.save(dashboard_path, dpi=150)
        
        print()
        print(f"💾 Dashboard saved to: {dashboard_path}")
        
        # Display in notebook
        import matplotlib.pyplot as plt
        plt.show()
        
        print()
        print("=" * 70)
        print("✅ DASHBOARD COMPLETE")
        print("=" * 70)
        print()
        
        if drift_data is not None:
            print("📊 Drift Analysis Summary:")
            print(f"   Status: {drift_data['status'].upper()}")
            print(f"   Metrics tracked: {len(drift_data['drift_scores'])}")
            print()
            print("   Drift scores:")
            for metric, score in drift_data['drift_scores'].items():
                status_emoji = '✅' if score < 0.1 else '⚠️ ' if score < 0.2 else '🚨'
                print(f"     {status_emoji} {metric}: {score:.4f}")
        
    except Exception as e:
        print()
        print(f"❌ Dashboard generation failed: {e}")
        print("   Check that metrics_df and drift_data are properly formatted")
        import traceback
        traceback.print_exc()
else:
    print()
    print("⚠️  Skipping dashboard generation - no metrics available")
    print("   Run training in Section 6 first")

In [ ]:
# Find best epoch based on validation loss
best_epoch_idx = metrics_df['val/loss'].idxmin()
best_epoch = metrics_df.loc[best_epoch_idx]

print("=" * 60)
print(" " * 20 + "BEST EPOCH ANALYSIS")
print("=" * 60)
print(f"{'Best Epoch:':<25} {int(best_epoch['epoch']) + 1}")
print(f"{'Validation Loss:':<25} {best_epoch['val/loss']:.4f}")
print(f"{'Validation Perplexity:':<25} {best_epoch['val/perplexity']:.2f}")
print(f"{'Training Loss:':<25} {best_epoch['train/loss']:.4f}")
print(f"{'Learning Rate:':<25} {best_epoch['train/learning_rate']:.2e}")
print("=" * 60)

# Load best checkpoint
best_checkpoint_path = f"{workspace_root}/checkpoints/{config.run_name}_epoch{int(best_epoch['epoch']) + 1}.pt"
if os.path.exists(best_checkpoint_path):
    print(f"\n💾 Best checkpoint: {best_checkpoint_path}")
else:
    print(f"\n⚠️ Best checkpoint not found (may not have been saved)")

In [ ]:
# Display metrics table
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

display_cols = ['epoch', 'train/loss', 'val/loss', 'val/perplexity', 'train/learning_rate']
available_cols = [col for col in display_cols if col in metrics_df.columns]

print("\nTraining Metrics Summary:")
print(metrics_df[available_cols].to_string(index=False))

# Export to CSV
csv_path = f'{workspace_root}/results/{config.run_name}_metrics.csv'
metrics_df.to_csv(csv_path, index=False)
print(f"\n✅ Metrics exported to: {csv_path}")

In [ ]:
import torch

if torch.cuda.is_available():
    print("=" * 60)
    print(" " * 20 + "GPU METRICS")
    print("=" * 60)

    gpu_cols = [col for col in metrics_df.columns if col.startswith('gpu/')]
    if gpu_cols:
        print(metrics_df[['epoch'] + gpu_cols].tail(5).to_string(index=False))

        # Plot GPU utilization
        import matplotlib.pyplot as plt
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

        if 'gpu/memory_allocated_mb' in metrics_df.columns:
            ax1.plot(metrics_df['epoch'], metrics_df['gpu/memory_allocated_mb'])
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('GPU Memory (MB)')
            ax1.set_title('GPU Memory Usage')
            ax1.grid(True)

        if 'gpu/utilization_percent' in metrics_df.columns:
            ax2.plot(metrics_df['epoch'], metrics_df['gpu/utilization_percent'])
            ax2.set_xlabel('Epoch')
            ax2.set_ylabel('GPU Utilization (%)')
            ax2.set_title('GPU Utilization')
            ax2.grid(True)

        plt.tight_layout()
        plt.savefig(f'{workspace_root}/results/{config.run_name}_gpu_metrics.png', dpi=100)
        plt.show()
        print(f"\n✅ GPU metrics saved")
    else:
        print("⚠️ No GPU metrics collected during training")
    print("=" * 60)
else:
    print("ℹ️ Training was performed on CPU (no GPU metrics available)")

<a id="section-8"></a>
# 💾 Section 8: Export & Results

Download checkpoints, configs, and results.

In [ ]:
import os
from google.colab import files

print("=" * 60)
print(" " * 20 + "EXPORT SUMMARY")
print("=" * 60)
print(f"\n📁 Workspace: {workspace_root}")
print(f"\n📊 Results:")
print(f"   - Dashboard: {config.run_name}_dashboard.png")
print(f"   - Metrics CSV: {config.run_name}_metrics.csv")
print(f"   - Config: {os.path.basename(config_path)}")
print(f"\n💾 Checkpoints:")

checkpoint_dir = f"{workspace_root}/checkpoints"
checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith(config.run_name)]
for ckpt in sorted(checkpoints):
    ckpt_path = os.path.join(checkpoint_dir, ckpt)
    size_mb = os.path.getsize(ckpt_path) / (1024 * 1024)
    print(f"   - {ckpt} ({size_mb:.1f} MB)")

print("=" * 60)

In [ ]:
# Download results to local machine
download_results = False  #@param {type:"boolean"}

if download_results:
    print("Downloading files...")

    # Download dashboard
    dashboard_file = f'{workspace_root}/results/{config.run_name}_dashboard.png'
    if os.path.exists(dashboard_file):
        files.download(dashboard_file)

    # Download metrics CSV
    metrics_file = f'{workspace_root}/results/{config.run_name}_metrics.csv'
    if os.path.exists(metrics_file):
        files.download(metrics_file)

    # Download config
    if os.path.exists(config_path):
        files.download(config_path)

    # Download best checkpoint
    if os.path.exists(best_checkpoint_path):
        files.download(best_checkpoint_path)
        print(f"✅ Downloaded {os.path.basename(best_checkpoint_path)}")

    print("✅ Downloads complete")
else:
    print("ℹ️ Downloads skipped. Files are saved in Google Drive.")
    print(f"   Access them at: {workspace_root}")

In [ ]:
# @title 📦 Production Export Bundle (v3.5) { display-mode: "form" }

if training_config.export_bundle:
    print("=" * 70)
    print("PRODUCTION EXPORT BUNDLE GENERATION (v3.5)")
    print("=" * 70)
    print()
    
    print("📦 Creating complete deployment bundle...")
    print(f"   Formats: {', '.join(training_config.export_formats)}")
    print(f"   Task: {task_spec.name}")
    print(f"   Modality: {task_spec.modality}")
    print()
    
    try:
        # Get best model from training results
        if 'results' in globals() and 'model' in globals():
            export_model = model
            print("✅ Using trained model for export")
        else:
            print("⚠️  No trained model found, using current model state")
            export_model = model if 'model' in globals() else None
        
        if export_model is not None:
            # Create export bundle
            print()
            print("🔧 Generating export artifacts...")
            print("   (This may take 1-2 minutes)")
            print()
            
            export_dir = create_export_bundle(
                model=export_model,
                config=config_obj if 'config_obj' in globals() else {},
                task_spec=task_spec,
                training_config=training_config,
                export_base_dir=f'{workspace_root}/exports'
            )
            
            print()
            print("=" * 70)
            print("✅ EXPORT BUNDLE CREATED SUCCESSFULLY!")
            print("=" * 70)
            print()
            print(f"📁 Location: {export_dir}")
            print()
            print("Bundle Contents:")
            print("-" * 70)
            print()
            print("  📁 artifacts/")
            
            for fmt in training_config.export_formats:
                if fmt == "onnx":
                    print("     ├── model.onnx (ONNX format)")
                elif fmt == "torchscript":
                    print("     ├── model.torchscript.pt (TorchScript)")
                elif fmt == "pytorch":
                    print("     ├── model.pytorch.pt (PyTorch state dict)")
            
            print()
            print("  📁 configs/")
            print("     ├── task_spec.json (task configuration)")
            print("     ├── training_config.json (training settings)")
            print("     └── torchserve_config.json (TorchServe deployment)")
            print()
            print("  📄 inference.py (standalone inference script)")
            print("  📄 README.md (quickstart guide)")
            print("  🐳 Dockerfile (container deployment)")
            print("  📋 requirements.txt (runtime dependencies)")
            print()
            print("-" * 70)
            print()
            print("QUICK START GUIDE")
            print("-" * 70)
            print()
            
            if task_spec.modality == "vision":
                print("Local Inference:")
                print(f"  cd {export_dir}")
                print(f"  python inference.py --input image.jpg --model artifacts/model.onnx")
                print()
                print("Docker Deployment:")
                print(f"  cd {export_dir}")
                print("  docker build -t model-inference .")
                print("  docker run -p 8080:8080 model-inference")
                print()
                print("Test the API:")
                print("  curl -X POST http://localhost:8080/predict \\")
                print("    -F 'image=@test_image.jpg'")
            else:
                print("Local Inference:")
                print(f"  cd {export_dir}")
                print(f"  python inference.py --input 'Your text here' --model artifacts/model.onnx")
                print()
                print("Docker Deployment:")
                print(f"  cd {export_dir}")
                print("  docker build -t model-inference .")
                print("  docker run -p 8080:8080 model-inference")
                print()
                print("Test the API:")
                print("  curl -X POST http://localhost:8080/predict \\")
                print("    -d '{\"text\": \"Your input text\"}'")
            
            print()
            print("=" * 70)
            print()
            print("💡 The export bundle includes everything needed for production:")
            print("   ✅ Optimized model formats (ONNX, TorchScript)")
            print("   ✅ Standalone inference script")
            print("   ✅ Docker container definition")
            print("   ✅ TorchServe deployment config")
            print("   ✅ Complete documentation")
            print()
            print(f"📦 Export bundle is ready in: {export_dir}")
            print()
            
        else:
            print("❌ No model available for export")
            print("   Please run training first (Section 6)")
    
    except Exception as e:
        print()
        print("❌ Export bundle generation failed!")
        print()
        print(f"Error: {e}")
        print()
        print("Troubleshooting:")
        print("  - Ensure model is properly trained")
        print("  - Check that all required configs are available")
        print("  - Verify export formats are supported")
        print()
        import traceback
        traceback.print_exc()

else:
    print("=" * 70)
    print("EXPORT BUNDLE DISABLED")
    print("=" * 70)
    print()
    print("ℹ️  Production export bundle generation is disabled")
    print()
    print("To enable:")
    print("  1. Go back to Section 4 (Training Configuration)")
    print("  2. Set export_bundle = True")
    print("  3. Re-run configuration and training")
    print()
    print("💡 Benefits of export bundles:")
    print("   - Production-ready deployment artifacts")
    print("   - Multiple model formats (ONNX, TorchScript, PyTorch)")
    print("   - Docker containerization")
    print("   - TorchServe integration")
    print("   - Complete documentation")
    print()
    print("=" * 70)
    print()

In [ ]:
# Compare with previous runs
all_runs = db.list_runs(limit=10)

if len(all_runs) > 1:
    print("=" * 60)
    print(" " * 15 + "COMPARISON WITH PREVIOUS RUNS")
    print("=" * 60)

    comparison_data = []
    for run in all_runs:
        comparison_data.append({
            'run_name': run.get('run_name', 'unknown'),
            'final_val_loss': run.get('metrics', {}).get('val/loss', float('nan')),
            'final_perplexity': run.get('metrics', {}).get('val/perplexity', float('nan')),
            'data_source': run.get('data_source', 'unknown'),
            'timestamp': run.get('timestamp', 'unknown')
        })

    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    print("=" * 60)
else:
    print("ℹ️ No previous runs to compare (this is your first run)")

<a id="section-9"></a>
# 🔬 Section 9: Advanced Features

Hyperparameter search, multi-run experiments, and optimization.

In [ ]:
from utils.tier3_training_utilities import test_hyperparameter_search

# Hyperparameter search configuration
run_hp_search = False  #@param {type:"boolean"}
n_trials = 10  #@param {type:"integer"}
search_timeout = 3600  #@param {type:"integer"}

if run_hp_search:
    print("🔍 Starting hyperparameter search...")
    print(f"   Trials: {n_trials}")
    print(f"   Timeout: {search_timeout}s ({search_timeout/60:.1f} min)")
    print("\n⚠️ This may take a while. Progress will be shown below.")

    # Define search space
    search_space = {
        'learning_rate': (1e-5, 1e-3),
        'batch_size': [4, 8, 16],
        'warmup_ratio': (0.0, 0.2),
        'weight_decay': (0.0, 0.1)
    }

    print(f"\nSearch space: {search_space}")
else:
    print("ℹ️ Hyperparameter search disabled")
    print("   Set 'run_hp_search = True' to enable")

In [ ]:
if run_hp_search:
    # Run search
    hp_results = test_hyperparameter_search(
        model=model,
        config=config,
        train_data=train_data,
        val_data=val_data,
        n_trials=n_trials,
        timeout=search_timeout,
        use_wandb=use_wandb
    )

    # Display results
    print("\n" + "=" * 60)
    print(" " * 15 + "HYPERPARAMETER SEARCH RESULTS")
    print("=" * 60)
    print(f"\nBest parameters:")
    for param, value in hp_results['best_params'].items():
        print(f"   {param}: {value}")

    print(f"\nBest validation loss: {hp_results['best_value']:.4f}")
    print(f"\nAll trials:")
    print(hp_results['trials_df'].to_string(index=False))

    # Save results
    hp_results['trials_df'].to_csv(
        f'{workspace_root}/results/{config.run_name}_hp_search.csv',
        index=False
    )
    print(f"\n✅ Results saved to: {config.run_name}_hp_search.csv")
    print("=" * 60)
else:
    print("⏭️ Hyperparameter search skipped")

## 🎉 Training Complete!

### Next Steps

1. **Review Results**: Check the dashboard in Section 6
2. **Download Files**: Use Section 7 to download checkpoints
3. **Compare Runs**: See Section 7 for comparison with previous experiments
4. **Optimize**: Try hyperparameter search in Section 8

### Workspace Structure

All files are saved in Google Drive:
```
/content/drive/MyDrive/TransformerTraining/
├── checkpoints/     # Model weights (.pt files)
├── configs/         # Training configs (.json files)
├── results/         # Dashboards, metrics, plots
├── datasets/        # Cached datasets
└── experiments.db   # SQLite tracking database
```

### Resources

- [Transformer Builder Documentation](https://transformer-builder.com/docs)
- [Training Utilities Reference](https://github.com/matt-hans/transformer-builder-colab-templates)
- [W&B Dashboard](https://wandb.ai) (if enabled)

---

**💡 Tip**: Save this notebook to Google Drive for future use!

In [ ]:
from utils.training import TrainingConfig, build_task_spec, build_eval_config, run_training
from utils.adapters import DecoderOnlyLMAdapter

cfg = TrainingConfig(epochs=1, batch_size=2, vocab_size=101, max_seq_len=16)

# Build Task & Eval configs
task = build_task_spec(cfg)
eval_cfg = build_eval_config(cfg)

# Choose adapter for your architecture
adapter = DecoderOnlyLMAdapter()

# Assumes `model` is already defined above
results = run_training(model, adapter, cfg, task, eval_cfg)
print("Eval summary:", results.get("eval_summary"))


In [ ]:
# Mode selection and config preview (v4.0.0)
from utils.ui.presets import build_configs_for_mode
mode = 'FAST_DEV'  # or 'STANDARD_EXPERIMENT', 'ABLATION_SWEEP'
training_cfg, task_spec, eval_cfg = build_configs_for_mode(mode)
print('Mode:', mode)
print('TrainingConfig epochs/batch_size:', training_cfg.epochs, training_cfg.batch_size)
print('Task:', task_spec.name, '| Eval dataset:', eval_cfg.dataset_id)


In [ ]:
# Simple grid sweep (learning_rate x num_layers)
from utils.training.sweep_runner import run_grid_sweep
from utils.training.experiment_db import ExperimentDB
from utils.adapters import DecoderOnlyLMAdapter
from utils.training.training_core import run_training
from utils.training import build_task_spec, build_eval_config

# Base config from mode (or build a fresh one)
base = training_cfg
sweep_id = 'demo_sweep_lr_depth'
param_grid = {
    'learning_rate': [5e-5, 1e-4],
    'num_layers': [2, 3],
}

db = ExperimentDB('experiments.db')
adapter = DecoderOnlyLMAdapter()

def run_fn(cfg):
    task = build_task_spec(cfg)
    ecfg = build_eval_config(cfg)
    # Log run
    run_id = db.log_run(
        run_name=f"sweep-{sweep_id}",
        config=cfg.to_dict(),
        notes='grid sweep',
        sweep_id=sweep_id,
        sweep_params={k: getattr(cfg, k) for k in param_grid.keys()},
    )
    # Execute training + tiny eval
    _ = run_training(model, adapter, cfg, task, ecfg)
    db.update_run_status(run_id, 'completed')
    return str(run_id)

run_ids = run_grid_sweep(base, param_grid, run_fn)
print('Completed runs:', run_ids)
print('Runs for sweep:')
print(db.get_runs_for_sweep(sweep_id))


In [ ]:
# Create a Repro Bundle (zip) for this run
from utils.training.export_utilities import create_repro_bundle
from utils.training.experiment_db import ExperimentDB
try:
    from utils.training.environment_snapshot import capture_environment
    env_info = capture_environment()
except Exception:
    env_info = {}

run_id = 'local_run'  # replace with actual run id if using ExperimentDB
zip_path = create_repro_bundle(
    run_id=run_id,
    training_config=training_cfg,
    task_spec=task_spec,
    eval_config=eval_cfg,
    environment_snapshot=env_info,
    experiment_db=ExperimentDB('experiments.db'),
    dashboard_paths=None,
    output_path='./repro'
)
print('Repro bundle created at:', zip_path)


In [ ]:
# Sweep visualization
from utils.training.experiment_db import ExperimentDB
sweep_id = 'demo_sweep_lr_depth'
db = ExperimentDB('experiments.db')
df = db.get_runs_for_sweep(sweep_id)
print(df)


In [ ]:
# Load model from GitHub Gist and log metadata
from utils.adapters.gist_loader import load_gist_model
from utils.training.experiment_db import ExperimentDB
from pathlib import Path
import importlib.util

gist_id = 'abcdef1234567890'  # replace
revision = None
md = load_gist_model(gist_id, revision)
print('Gist SHA:', md.sha256)
root = Path('./external/gists') / md.gist_id / (md.revision or 'latest')
model_path = root / 'model.py'
if model_path.exists():
    spec = importlib.util.spec_from_file_location('gist_model', str(model_path))
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    if hasattr(mod, 'build_model'):
        model = mod.build_model()
        print('Loaded model using build_model()')
    elif hasattr(mod, 'Model'):
        model = mod.Model()
        print('Loaded model using Model class')
    else:
        print('No known model entrypoint; define model manually')

# Log to ExperimentDB
try:
    db = ExperimentDB('experiments.db')
    run_id = db.log_run(
        run_name='gist-training',
        config=training_cfg.to_dict(),
        notes='Gist-based run',
        gist_id=md.gist_id,
        gist_revision=md.revision,
        gist_sha256=md.sha256,
    )
    print('Run logged:', run_id)
except Exception as e:
    print('DB logging skipped:', e)


In [ ]:
# Log run to ExperimentDB and create a repro bundle for this training
from utils.training.experiment_db import ExperimentDB
from utils.training.export_utilities import create_repro_bundle
try:
    from utils.training.environment_snapshot import capture_environment
    env_info = capture_environment()
except Exception:
    env_info = {}

# Log run
db = ExperimentDB('experiments.db')
run_id = db.log_run(
    run_name='notebook-single-run',
    config=training_cfg.to_dict(),
    notes='single run from notebook'
)

# Train + tiny eval
nb_results = run_training(model, adapter, training_cfg, task_spec, eval_cfg)

# Mark run complete
db.update_run_status(run_id, 'completed')

# Create repro bundle
zip_path = create_repro_bundle(
    run_id=str(run_id),
    training_config=training_cfg,
    task_spec=task_spec,
    eval_config=eval_cfg,
    environment_snapshot=env_info,
    experiment_db=db,
    dashboard_paths=None,
    output_path='./repro'
)
print('Repro bundle zip:', zip_path)


In [ ]:
# Visualize sweep results (matplotlib)
import json
from utils.training.experiment_db import ExperimentDB
try:
    import matplotlib.pyplot as plt
except Exception:
    plt = None

sweep_id = 'demo_sweep_lr_depth'  # must match the sweep cell above

db = ExperimentDB('experiments.db')
df = db.get_runs_for_sweep(sweep_id)
if df.empty:
    print('No runs for sweep:', sweep_id)
else:
    # Parse sweep_params JSON strings
    params = df['sweep_params'].apply(lambda s: json.loads(s) if isinstance(s, str) and s else {})
    lrs = params.apply(lambda d: d.get('learning_rate', None))
    depths = params.apply(lambda d: d.get('num_layers', None))

    if plt is None:
        print('matplotlib not available; printing table only')
        print(df[['run_id', 'run_name', 'sweep_params']])
    else:
        # Bar plot: runs per learning_rate
        by_lr = lrs.value_counts().sort_index()
        plt.figure(figsize=(5,3))
        by_lr.plot(kind='bar', title='Runs per learning_rate')
        plt.ylabel('runs')
        plt.tight_layout()
        plt.show()

        # Scatter: num_layers by run index
        try:
            x = list(range(len(df)))
            y = depths.astype(float).fillna(0)
            plt.figure(figsize=(5,3))
            plt.scatter(x, y)
            plt.title('num_layers by run index')
            plt.xlabel('run index')
            plt.ylabel('num_layers')
            plt.tight_layout()
            plt.show()
        except Exception as e:
            print('Plotting skipped:', e)
